In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [2]:
Books = pd.read_csv('Books.csv')
Users = pd.read_csv('Users.csv')
Ratings = pd.read_csv('Ratings.csv')
UserRatings = pd.merge(Users,Ratings, on='User-ID')
df = pd.merge(UserRatings,Books, on='ISBN')
df

C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2945397334.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  Books = pd.read_csv('Books.csv')


,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,2,"stockton, california, usa",18.0,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,8,"timmins, ontario, canada",NaN,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,11400,"ottawa, ontario, canada",49.0,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
3,11676,"n/a, n/a, n/a",NaN,0002005018,8,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
4,41385,"sudbury, ontario, canada",NaN,0002005018,0,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
1031131,278851,"dallas, texas, usa",33.0,0743203763,0,As Hogan Said . . . : The 389 Best Things Anyo...,Randy Voorhees,2000,Simon &amp; Schuster,http://images.amazon.com/images/P/0743203763.0...,http://images.amazon.com/images/P/0743203763.0...,http://images.amazon.com/images/P/0743203763.0...
1031132,278851,"dallas, texas, usa",33.0,0767907566,5,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,http://images.amazon.com/images/P/0767907566.0...,http://images.amazon.com/images/P/0767907566.0...
1031133,278851,"dallas, texas, usa",33.0,0884159221,7,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,http://images.amazon.com/images/P/0884159221.0...,http://images.amazon.com/images/P/0884159221.0...
1031134,278851,"dallas, texas, usa",33.0,0912333022,7,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,http://images.amazon.com/images/P/0912333022.0...,http://images.amazon.com/images/P/0912333022.0...


In [3]:
df['Age'].fillna(df['Age'].mean(), inplace=True)

reader = Reader(rating_scale=(1, 10))


In [4]:
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()

model.fit(trainset)

predictions = model.test(testset)

accuracy.rmse(predictions)


RMSE: 3.4970


3.4970202421572094

In [5]:
#collaborative filtering
def get_book_recommendations(user_id, df, model, n_recommendations=5):
    """Generate top-n book recommendations for a given user based on collaborative filtering."""
    user_books = df[df['User-ID'] == user_id]
    other_books = df[~df['ISBN'].isin(user_books['ISBN'])]
    
    other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
    
    recommendations = other_books[['Book-Title', 'Book-Author', 'Estimated-Rating', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']].sort_values(by='Estimated-Rating', ascending=False).head(n_recommendations)
    return recommendations

In [6]:
user_id = 2
recommendations = get_book_recommendations(user_id, df, model, n_recommendations=5)
print(recommendations)

C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


       Book-Title   Book-Author  Estimated-Rating  \
160707       Free  Paul Vincent          7.633781   
160715       Free  Paul Vincent          7.633781   
160741       Free  Paul Vincent          7.633781   
160742       Free  Paul Vincent          7.633781   
160743       Free  Paul Vincent          7.633781   

                                              Image-URL-S  \
160707  http://images.amazon.com/images/P/1844262553.0...   
160715  http://images.amazon.com/images/P/1844262553.0...   
160741  http://images.amazon.com/images/P/1844262553.0...   
160742  http://images.amazon.com/images/P/1844262553.0...   
160743  http://images.amazon.com/images/P/1844262553.0...   

                                              Image-URL-M  \
160707  http://images.amazon.com/images/P/1844262553.0...   
160715  http://images.amazon.com/images/P/1844262553.0...   
160741  http://images.amazon.com/images/P/1844262553.0...   
160742  http://images.amazon.com/images/P/1844262553.0...   
160743  h

In [7]:
import numpy as np

def age_group(age):
    if age < 18:
        return 'Teen'
    elif 18 <= age <= 35:
        return 'Young Adult'
    elif 35 < age <= 50:
        return 'Adult'
    else:
        return 'Senior'


In [8]:
df['Age-Group'] = df['Age'].apply(age_group)

In [9]:
#by age and location
def get_recommendations_by_age_location(user_id, df, model, n_recommendations=5):
    
    user_data = df[df['User-ID'] == user_id]
    
    if user_data.empty:
        print("User not found!")
        return None

    user_location = user_data['Location'].values[0]
    user_age_group = user_data['Age-Group'].values[0]
    
    print(f"User Location: {user_location}, User Age Group: {user_age_group}")
    
    other_books = df[~df['ISBN'].isin(user_data['ISBN'])]
    
    filtered_books = other_books[(other_books['Location'] == user_location) & 
                                 (other_books['Age-Group'] == user_age_group)]
    
    if len(filtered_books) < n_recommendations:
        filtered_books = other_books
    
    filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
    
    recommendations = filtered_books[['Book-Title', 'Book-Author', 'Estimated-Rating', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']].sort_values(by='Estimated-Rating', ascending=False).head(n_recommendations)
    
    return recommendations

In [10]:
user_id = 2
recommendations_age_location = get_recommendations_by_age_location(user_id, df, model, n_recommendations=5)
print(recommendations_age_location)

User Location: stockton, california, usa, User Age Group: Young Adult
                                               Book-Title    Book-Author  \
186980                The Clan of the Cave Bear : a novel   Jean M. Auel   
9286                   The Five People You Meet in Heaven    Mitch Albom   
277120                            The Secret Life of Bees  Sue Monk Kidd   
54455   A Child Called \It\": One Child's Courage to S...    Dave Pelzer   
91586   Valley of the Horses (Auel, Jean M. , Earth's ...   Jean M. Auel   

        Estimated-Rating                                        Image-URL-S  \
186980          5.024893  http://images.amazon.com/images/P/0553148001.0...   
9286            5.011109  http://images.amazon.com/images/P/0786868716.0...   
277120          4.770473  http://images.amazon.com/images/P/0670894605.0...   
54455           4.762982  http://images.amazon.com/images/P/1558743669.0...   
91586           4.515017  http://images.amazon.com/images/P/0553234811.0...   

C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
from sklearn.neighbors import NearestNeighbors
import numpy as np


In [15]:
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
#content based reccomendation
def get_content_based_recommendations_ann(user_id, df, nn_model, tfidf_matrix, n_recommendations=5):
    
    user_data = df[df['User-ID'] == user_id]
    
    if user_data.empty:
        print("User not found!")
        return None
    
    highest_rated_book_index = user_data.index[0]
    
    distances, indices = nn_model.kneighbors(tfidf_matrix[highest_rated_book_index], n_neighbors=n_recommendations+1)
    
    similar_books_indices = indices.flatten()[1:] 
    
    recommendations = df.iloc[similar_books_indices]
    
    return recommendations[['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']]


In [17]:
user_id = 2
content_recommendations_ann = get_content_based_recommendations_ann(user_id, df, nn_model, tfidf_matrix, n_recommendations=5)
print(content_recommendations_ann)

                                                Book-Title  \
687050                                 Classical mythology   
687051                                 Classical mythology   
917564                                 Classical mythology   
1026517                                Classical Mythology   
925830   Who's Who in Classical Mythology (Who's Who Se...   

                Book-Author                Publisher  \
687050     Mark P.O Morford                  Longman   
687051     Mark P.O Morford                  Longman   
917564    Mark P. O Morford                  Longman   
1026517  Mark P. O. Morford    John Wiley &amp; Sons   
925830        Michael Grant  Oxford University Press   

                                               Image-URL-S  \
687050   http://images.amazon.com/images/P/0582280044.0...   
687051   http://images.amazon.com/images/P/0582280044.0...   
917564   http://images.amazon.com/images/P/0801304652.0...   
1026517  http://images.amazon.com/images/P

In [18]:
book_popularity = df.groupby('Book-Title').agg(
    avg_rating=('Book-Rating', 'mean'),      
    num_ratings=('Book-Rating', 'count')     
).reset_index()
popular_books = book_popularity.sort_values(by=['num_ratings', 'avg_rating'], ascending=False)


In [19]:
def get_popularity_based_recommendations(df, popular_books, n_recommendations=5):
    """Generate top-n popularity-based book recommendations."""
    
    recommendations = pd.merge(popular_books, df[['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']].drop_duplicates(), 
                               on='Book-Title')
    
    return recommendations[['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']].head(n_recommendations)

popularity_recommendations = get_popularity_based_recommendations(df, popular_books, n_recommendations=5)
print(popularity_recommendations)

                  Book-Title   Book-Author           Publisher  \
0                Wild Animus  Rich Shapero             Too Far   
1  The Lovely Bones: A Novel  Alice Sebold       Little, Brown   
2          The Da Vinci Code     Dan Brown           Doubleday   
3          The Da Vinci Code     DAN BROWN  Random House Audio   
4          The Da Vinci Code     DAN BROWN  Random House Audio   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0971880107.0...   
1  http://images.amazon.com/images/P/0316666343.0...   
2  http://images.amazon.com/images/P/0385504209.0...   
3  http://images.amazon.com/images/P/0739307312.0...   
4  http://images.amazon.com/images/P/0739302043.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0971880107.0...   
1  http://images.amazon.com/images/P/0316666343.0...   
2  http://images.amazon.com/images/P/0385504209.0...   
3  http://images.amazon.com/images/P/07393

In [20]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate

In [21]:
reader = Reader(rating_scale=(1, 10)) 

data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

svd_model = SVD()

svd_model.fit(trainset)

predictions = svd_model.test(testset)

accuracy.rmse(predictions)

RMSE: 3.4921


3.4921429354122058

In [22]:
# recommendations using matrix factorization (SVD)
def get_matrix_factorization_recommendations(user_id, df, svd_model, n_recommendations=5):
    all_books = df['ISBN'].unique()
    
    user_rated_books = df[df['User-ID'] == user_id]['ISBN'].tolist()
    
    predictions = []
    for isbn in all_books:
        if isbn not in user_rated_books:  
            prediction = svd_model.predict(user_id, isbn)
            predictions.append((isbn, prediction.est))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    top_n_isbns = [pred[0] for pred in predictions[:n_recommendations]]
    
    recommendations = df[df['ISBN'].isin(top_n_isbns)][['Book-Title', 'Book-Author', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']].drop_duplicates()
    
    return recommendations



In [23]:
user_id = 2
matrix_factorization_recommendations = get_matrix_factorization_recommendations(user_id, df, svd_model, n_recommendations=5)
print(matrix_factorization_recommendations)

                                               Book-Title  \
33024        Harry Potter and the Goblet of Fire (Book 4)   
33218   Harry Potter and the Order of the Phoenix (Boo...   
43323                                   The Little Prince   
160697                                               Free   
189658                                  All-American Girl   

                      Book-Author           Publisher  \
33024               J. K. Rowling          Scholastic   
33218               J. K. Rowling          Scholastic   
43323   Antoine de Saint-ExupÃ©ry            Harcourt   
160697               Paul Vincent  Upfront Publishing   
189658                  Meg Cabot       HarperCollins   

                                              Image-URL-S  \
33024   http://images.amazon.com/images/P/0439139597.0...   
33218   http://images.amazon.com/images/P/043935806X.0...   
43323   http://images.amazon.com/images/P/0156528207.0...   
160697  http://images.amazon.com/images/P/1844

In [27]:
recommendations_for_first_100 = []

In [28]:
for user_id in df['User-ID'].unique()[:100]:
    matrix_rec = get_matrix_factorization_recommendations(user_id, df, svd_model, n_recommendations=5)
    content_rec = get_content_based_recommendations_ann(user_id, df, nn_model, tfidf_matrix, n_recommendations=5)
    popularity_rec = get_popularity_based_recommendations(df, popular_books, n_recommendations=5)
    agelocation_rec = get_recommendations_by_age_location(user_id, df, model, n_recommendations=5)
    collaborativefiltering_rec = get_book_recommendations(user_id, df, model, n_recommendations=5)
    
    recommendations_for_first_100.append({
        'User-ID': user_id,
        'Matrix Factorization Recommendations': matrix_rec.to_dict(orient='records'),
        'Content-Based Recommendations': content_rec.to_dict(orient='records'),
        'Popularity-Based Recommendations': popularity_rec.to_dict(orient='records'),
        'Age-Location Based Recommendations': agelocation_rec.to_dict(orient='records'),
        'Collaborative-Filtering Based Recommendations': collaborativefiltering_rec.to_dict(orient='records')
    })

User Location: stockton, california, usa, User Age Group: Young Adult


C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


User Location: timmins, ontario, canada, User Age Group: Adult


C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


User Location: ottawa, ontario, canada, User Age Group: Adult


...

C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


User Location: mechanicsville, maryland, usa, User Age Group: Adult


C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


User Location: pinckney, michigan, usa, User Age Group: Adult


C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2262779235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Estimated-Rating'] = filtered_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)
C:\Users\dheep\AppData\Local\Temp\ipykernel_28308\2408220243.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_books['Estimated-Rating'] = other_books['ISBN'].apply(lambda x: model.predict(user_id, x).est)


In [29]:
recommendations_df = pd.DataFrame(recommendations_for_first_100)

recommendations_df.head(10)

,User-ID,Matrix Factorization Recommendations,Content-Based Recommendations,Popularity-Based Recommendations,Age-Location Based Recommendations,Collaborative-Filtering Based Recommendations
0,2,[{'Book-Title': 'Harry Potter and the Goblet o...,"[{'Book-Title': 'Classical mythology', 'Book-A...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...",[{'Book-Title': 'The Clan of the Cave Bear : a...,"[{'Book-Title': 'Free', 'Book-Author': 'Paul V..."
1,8,[{'Book-Title': 'The Fellowship of the Ring (T...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...",[{'Book-Title': 'The Stand (The Complete and U...,"[{'Book-Title': 'Free', 'Book-Author': 'Paul V..."
2,11400,"[{'Book-Title': 'The Da Vinci Code', 'Book-Aut...","[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...","[{'Book-Title': 'The Saving Graces: A Novel', ...","[{'Book-Title': 'From Potter's Field', 'Book-A..."
3,11676,[{'Book-Title': 'Corelli's Mandolin : A Novel'...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...",[{'Book-Title': 'The Women of Brewster Place (...,[{'Book-Title': 'The Sixteen Pleasures: A Nove...
4,41385,[{'Book-Title': 'Harry Potter and the Goblet o...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...",[{'Book-Title': 'The Red Tent (Bestselling Bac...,"[{'Book-Title': 'Free', 'Book-Author': 'Paul V..."
5,67544,[{'Book-Title': 'Seabiscuit: An American Legen...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...","[{'Book-Title': 'The Catcher in the Rye', 'Boo...","[{'Book-Title': 'Free', 'Book-Author': 'Paul V..."
6,85526,[{'Book-Title': 'Left Behind: A Novel of the E...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...","[{'Book-Title': 'The Bourne Identity', 'Book-A...","[{'Book-Title': 'The Hot Zone', 'Book-Author':..."
7,96054,[{'Book-Title': 'Bridget Jones: The Edge of Re...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...","[{'Book-Title': 'Empire Falls', 'Book-Author':...","[{'Book-Title': 'Girl with a Pearl Earring', '..."
8,116866,"[{'Book-Title': 'The Beach House', 'Book-Autho...","[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...",[{'Book-Title': 'What Dreams May Come : A Nove...,"[{'Book-Title': 'SEAT OF THE SOUL', 'Book-Auth..."
9,123629,[{'Book-Title': 'Harry Potter and the Prisoner...,"[{'Book-Title': 'Clara Callan', 'Book-Author':...","[{'Book-Title': 'Wild Animus', 'Book-Author': ...","[{'Book-Title': 'To Kill a Mockingbird', 'Book...","[{'Book-Title': 'Free', 'Book-Author': 'Paul V..."
